## Introduction 

Pelajaran ini akan merangkumi: 
- Apa itu pemanggilan fungsi dan kegunaannya 
- Cara membuat panggilan fungsi menggunakan OpenAI 
- Cara mengintegrasikan panggilan fungsi ke dalam aplikasi 

## Learning Goals 

Selepas menamatkan pelajaran ini anda akan tahu cara dan memahami: 

- Tujuan menggunakan pemanggilan fungsi 
- Menyediakan Panggilan Fungsi menggunakan Perkhidmatan OpenAI 
- Mereka bentuk panggilan fungsi yang berkesan untuk kes penggunaan aplikasi anda 


## Memahami Panggilan Fungsi

Untuk pelajaran ini, kami ingin membina ciri untuk startup pendidikan kami yang membolehkan pengguna menggunakan chatbot untuk mencari kursus teknikal. Kami akan mengesyorkan kursus yang sesuai dengan tahap kemahiran mereka, peranan semasa dan teknologi yang diminati.

Untuk melengkapkan ini, kami akan menggunakan gabungan:
 - `OpenAI` untuk mencipta pengalaman sembang untuk pengguna
 - `Microsoft Learn Catalog API` untuk membantu pengguna mencari kursus berdasarkan permintaan pengguna
 - `Function Calling` untuk mengambil pertanyaan pengguna dan menghantarnya ke fungsi untuk membuat permintaan API.

Untuk memulakan, mari kita lihat mengapa kita ingin menggunakan panggilan fungsi pada mulanya:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # dapatkan respons baru dari GPT di mana ia boleh melihat respons fungsi


print(second_response.choices[0].message)


### Mengapa Panggilan Fungsi

Jika anda telah menyelesaikan mana-mana pelajaran lain dalam kursus ini, anda mungkin memahami kuasa menggunakan Model Bahasa Besar (LLM). Diharapkan anda juga dapat melihat beberapa hadnya.

Panggilan Fungsi adalah ciri Perkhidmatan OpenAI yang direka untuk menangani cabaran berikut:

Format Respons Tidak Konsisten:
- Sebelum panggilan fungsi, respons daripada model bahasa besar adalah tidak berstruktur dan tidak konsisten. Pembangun terpaksa menulis kod pengesahan yang kompleks untuk mengendalikan setiap variasi dalam output.

Integrasi Terhad dengan Data Luaran:
- Sebelum ciri ini, sukar untuk menggabungkan data dari bahagian lain aplikasi ke dalam konteks sembang.

Dengan menstandardkan format respons dan membolehkan integrasi lancar dengan data luaran, panggilan fungsi memudahkan pembangunan dan mengurangkan keperluan logik pengesahan tambahan.

Pengguna tidak dapat mendapatkan jawapan seperti "Apakah cuaca semasa di Stockholm?". Ini kerana model terhad kepada masa data dilatih.

Mari kita lihat contoh di bawah yang menggambarkan masalah ini:

Katakan kita ingin membuat pangkalan data data pelajar supaya kita boleh mencadangkan kursus yang sesuai kepada mereka. Di bawah ini kita mempunyai dua penerangan pelajar yang sangat serupa dalam data yang mereka kandungi.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Kami ingin menghantar ini kepada LLM untuk mengurai data. Ini boleh digunakan kemudian dalam aplikasi kami untuk menghantar ini ke API atau menyimpannya dalam pangkalan data.

Mari kita cipta dua arahan yang sama yang kami arahkan kepada LLM mengenai maklumat yang kami minati:


Kami ingin menghantar ini kepada LLM untuk menganalisis bahagian yang penting kepada produk kami. Jadi kami boleh mencipta dua arahan yang sama untuk mengarahkan LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Selepas mencipta dua arahan ini, kami akan menghantarnya ke LLM menggunakan `openai.ChatCompletion`. Kami menyimpan arahan dalam pembolehubah `messages` dan menetapkan peranan kepada `user`. Ini adalah untuk meniru mesej daripada pengguna yang ditulis kepada chatbot.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Sekarang kita boleh menghantar kedua-dua permintaan kepada LLM dan memeriksa respons yang kita terima.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Walaupun arahan adalah sama dan penerangan adalah serupa, kita boleh mendapat format yang berbeza bagi sifat `Grades`.

Jika anda menjalankan sel di atas beberapa kali, formatnya boleh menjadi `3.7` atau `3.7 GPA`.

Ini kerana LLM mengambil data tidak berstruktur dalam bentuk arahan bertulis dan juga mengembalikan data tidak berstruktur. Kita perlu mempunyai format berstruktur supaya kita tahu apa yang dijangka apabila menyimpan atau menggunakan data ini.

Dengan menggunakan pemanggilan fungsi, kita boleh memastikan bahawa kita menerima data berstruktur kembali. Apabila menggunakan pemanggilan fungsi, LLM sebenarnya tidak memanggil atau menjalankan sebarang fungsi. Sebaliknya, kita mencipta satu struktur untuk LLM ikuti bagi responsnya. Kita kemudian menggunakan respons berstruktur itu untuk mengetahui fungsi mana yang perlu dijalankan dalam aplikasi kita.


![Rajah Aliran Panggilan Fungsi](../../../../translated_images/Function-Flow.083875364af4f4bb.ms.png)


Kita kemudian boleh mengambil apa yang dikembalikan dari fungsi tersebut dan menghantarnya kembali ke LLM. LLM kemudian akan memberi respons menggunakan bahasa semula jadi untuk menjawab pertanyaan pengguna.


### Kes Penggunaan untuk menggunakan panggilan fungsi

**Memanggil Alat Luaran**  
Chatbot sangat baik dalam memberikan jawapan kepada soalan daripada pengguna. Dengan menggunakan panggilan fungsi, chatbot boleh menggunakan mesej daripada pengguna untuk melengkapkan tugas tertentu. Contohnya, seorang pelajar boleh meminta chatbot untuk "Hantar emel kepada pengajar saya mengatakan saya memerlukan lebih banyak bantuan dengan subjek ini". Ini boleh membuat panggilan fungsi kepada `send_email(to: string, body: string)`

**Mewujudkan Pertanyaan API atau Pangkalan Data**  
Pengguna boleh mencari maklumat menggunakan bahasa semula jadi yang ditukar menjadi pertanyaan berformat atau permintaan API. Contohnya ialah seorang guru yang meminta "Siapakah pelajar yang telah menyiapkan tugasan terakhir" yang boleh memanggil fungsi bernama `get_completed(student_name: string, assignment: int, current_status: string)`

**Mewujudkan Data Berstruktur**  
Pengguna boleh mengambil sekeping teks atau CSV dan menggunakan LLM untuk mengekstrak maklumat penting daripadanya. Contohnya, seorang pelajar boleh menukar artikel Wikipedia tentang perjanjian damai untuk membuat kad imbas AI. Ini boleh dilakukan dengan menggunakan fungsi yang dipanggil `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Membuat Panggilan Fungsi Pertama Anda

Proses membuat panggilan fungsi merangkumi 3 langkah utama:  
1. Memanggil API Chat Completions dengan senarai fungsi anda dan mesej pengguna  
2. Membaca respons model untuk melaksanakan tindakan iaitu menjalankan fungsi atau Panggilan API  
3. Membuat panggilan lain ke API Chat Completions dengan respons dari fungsi anda untuk menggunakan maklumat itu bagi menghasilkan respons kepada pengguna.


![Aliran Panggilan Fungsi](../../../../translated_images/LLM-Flow.3285ed8caf4796d7.ms.png)


### Elemen panggilan fungsi

#### Input Pengguna

Langkah pertama adalah untuk mencipta mesej pengguna. Ini boleh ditetapkan secara dinamik dengan mengambil nilai daripada input teks atau anda boleh menetapkan nilai di sini. Jika ini adalah kali pertama anda bekerja dengan API Chat Completions, kita perlu mentakrifkan `role` dan `content` mesej tersebut.

`role` boleh sama ada `system` (mencipta peraturan), `assistant` (model) atau `user` (pengguna akhir). Untuk panggilan fungsi, kita akan menetapkannya sebagai `user` dan satu contoh soalan.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Membuat fungsi.

Seterusnya kita akan mentakrifkan satu fungsi dan parameter fungsi tersebut. Kita akan menggunakan hanya satu fungsi di sini yang dipanggil `search_courses` tetapi anda boleh membuat pelbagai fungsi.

**Penting** : Fungsi-fungsi dimasukkan dalam mesej sistem kepada LLM dan akan dimasukkan dalam jumlah token yang anda ada.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definisi** 

Struktur definisi fungsi mempunyai pelbagai peringkat, setiap satu dengan sifatnya sendiri. Berikut adalah pecahan struktur bersarang:

**Sifat Fungsi Tahap Atas:**

`name` - Nama fungsi yang ingin kita panggil. 

`description` - Ini adalah penerangan tentang bagaimana fungsi berfungsi. Di sini penting untuk menjadi spesifik dan jelas 

`parameters` - Senarai nilai dan format yang anda mahu model hasilkan dalam responsnya 

**Sifat Objek Parameter:**

`type` - Jenis data objek parameter (biasanya "object")

`properties` - Senarai nilai khusus yang akan digunakan model untuk responsnya 

**Sifat Parameter Individu:**

`name` - Ditakrifkan secara tersirat oleh kunci sifat (contohnya, "role", "product", "level")

`type` - Jenis data parameter khusus ini (contohnya, "string", "number", "boolean") 

`description` - Penerangan tentang parameter khusus 

**Sifat Pilihan:**

`required` - Senarai parameter yang diperlukan untuk panggilan fungsi dapat diselesaikan 


### Membuat panggilan fungsi  
Selepas mentakrifkan fungsi, kita kini perlu memasukkannya dalam panggilan ke API Chat Completion. Kita lakukan ini dengan menambah `functions` ke dalam permintaan. Dalam kes ini `functions=functions`.  

Terdapat juga pilihan untuk menetapkan `function_call` kepada `auto`. Ini bermakna kita akan membiarkan LLM memutuskan fungsi mana yang harus dipanggil berdasarkan mesej pengguna dan bukannya menetapkannya sendiri.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Sekarang mari kita lihat respons dan bagaimana ia diformatkan:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Anda boleh lihat bahawa nama fungsi dipanggil dan daripada mesej pengguna, LLM dapat mencari data untuk memenuhi argumen fungsi tersebut.


## 3.Mengintegrasikan Panggilan Fungsi ke dalam Aplikasi. 


Selepas kita menguji respons yang diformat dari LLM, kini kita boleh mengintegrasikannya ke dalam aplikasi. 

### Menguruskan aliran 

Untuk mengintegrasikan ini ke dalam aplikasi kita, mari ambil langkah-langkah berikut: 

Pertama, mari buat panggilan ke perkhidmatan OpenAI dan simpan mesej dalam pembolehubah yang dipanggil `response_message`. 


In [ ]:
response_message = response.choices[0].message

Sekarang kita akan mentakrifkan fungsi yang akan memanggil API Microsoft Learn untuk mendapatkan senarai kursus:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Sebagai amalan terbaik, kita akan melihat sama ada model ingin memanggil fungsi. Selepas itu, kita akan mencipta salah satu fungsi yang tersedia dan memadankannya dengan fungsi yang sedang dipanggil.  
Kita kemudian akan mengambil argumen fungsi tersebut dan memetakannya kepada argumen dari LLM.

Akhir sekali, kita akan menambah mesej panggilan fungsi dan nilai-nilai yang dikembalikan oleh mesej `search_courses`. Ini memberikan LLM semua maklumat yang diperlukan untuk  
memberi respons kepada pengguna menggunakan bahasa semula jadi.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Sekarang kami akan menghantar mesej yang dikemas kini kepada LLM supaya kami dapat menerima respons bahasa semula jadi dan bukannya respons berformat JSON API.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Cabaran Kod

Kerja yang hebat! Untuk meneruskan pembelajaran anda mengenai Panggilan Fungsi OpenAI anda boleh membina: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - Lebih banyak parameter fungsi yang mungkin membantu pelajar mencari lebih banyak kursus. Anda boleh mencari parameter API yang tersedia di sini:  
 - Cipta satu lagi panggilan fungsi yang mengambil lebih banyak maklumat daripada pelajar seperti bahasa ibunda mereka  
 - Cipta pengendalian ralat apabila panggilan fungsi dan/atau panggilan API tidak mengembalikan sebarang kursus yang sesuai  


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk ketepatan, sila ambil maklum bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang sahih. Untuk maklumat penting, terjemahan profesional oleh manusia adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
